# Aufgabe 1 - k-NN Klassifikation

## Teilaufgabe a)

## Teilaufgabe b)

## Teilaufgabe c)

In [2]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist # Vektorisierte Funktion,
#um schnell den Abstand zwischen einer Menge an Punkten zu bestimmen und in einer Matrix zu speichern
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [3]:
class KNN:
    '''KNN Classifier.

    Attributes
    ----------
    k : int
        Number of neighbors to consider.
    '''
    def __init__(self, k):
        '''Initialization.
        Parameters are stored as member variables/attributes.
        
        Parameters
        ----------
        k : int
            Number of neighbors to consider.
        '''
        self.k = k

    def fit(self, X, y):
        '''Fit routine.
        Training data is stored within object.
        
        Parameters
        ----------
        X : numpy.array, shape=(n_samples, n_attributes)
            Training data.
        y : numpy.array shape=(n_samples)
            Training labels.
        '''
        self.training_data = X
        self.training_labels = y

    def predict(self, X):
        '''Prediction routine.
        Predict class association of each sample of X.
        
        Parameters
        ----------
        X : numpy.array, shape=(n_samples, n_attributes)
            Data to classify.
        
        Returns
        -------
        prediction : numpy.array, shape=(n_samples)
            Predictions, containing the predicted label of each sample.
        '''
        
        # First we need to find a few parameters
        n_attributes, n_samples = X.shape
        n_training_data = np.size(self.training_data, axis = 1) # this means finding the amount of columns
        
        prediction = []
        
        for i in range(X.shape[0]): # für jeden zu klassifizierenden Datenpunkt
            back = 0 # ein Counter für die Anzahl der nächsten Nachbarn, die Background-Ereignisse sind
            sig = 0 # und nochmal für die Signal-nächsten-Nachbarn
            a = X[i] # keine Veränderung von Arrays in For Loops in Python, deswegen ein temporäres Objekt
            if len(a.shape) != 2: # mache eben aus a ein 2d-Array mit nur einer Zeile, falls es noch keins ist.
                a = a.reshape(1, a.shape[0])
            distanceTest = cdist(a, self.training_data)[0] # Hierfür MUESSEN beide Arrays zweidimensional sein
            distanceList = np.stack((distanceTest, self.training_labels), axis=-1) # Klebe die beiden Arrays als Spalten zusammen!
            sortedDistanceList = distanceList[distanceList[:,0].argsort()] # Sortiere nach der ersten Spalte!
            kNearestNeighbors = sortedDistanceList[:self.k] # Nimm nur die ersten k Werte -> k nächste Nachbarn
            for l in range(0, self.k):
                if kNearestNeighbors[l,1] == 0:
                    back += 1
                else:
                    sig +=1
            if back > sig:
                prediction.append(0)
            elif back < sig:
                prediction.append(1)
            else:
                prediction.append(float('NaN')) # Falls gleich viele Signale und Background in der Nachbarschaft liegen
                # können wir keine Entscheidung treffen...
        return np.asarray(prediction) # Gib ein numpy Array zurück, weil man damit besser arbeiten kann als mit einer Liste

## Teilaufgabe d)
Der selbst geschriebene KNN-Klassifikator wird an dem NeutrinoMC-Beispiel getestet.
Wir definieren zunächst einige Funktionen und analysieren die Daten. Die Ergebnisse sind zufriedenstellend, KNN kann die Daten recht gut zuordnen.

In [4]:
def getDataFromFile(filename, key, log = False):
    '''Gets the hits, x and y from a NeutrinoMC-file, cuts any nans and returns them as a numpy matrix
    If getDataFromFile is invoked with log = True the decadic logarithm of the hits will be used.'''
    hdf = pd.read_hdf(filename, key)
    hits = hdf.NumberOfHits
    x = hdf.x
    y = hdf.y
    hits = np.asarray(hits)
    if log == True:
        hits = np.log10(hits)
    hits = hits[~np.isnan(hits)] # ~ means not 
    x = np.asarray(x)
    x = x[~np.isnan(x)]
    y = np.asarray(y)
    y = y[~np.isnan(y)]
    return np.transpose(np.matrix([hits, x, y])) # Wir wollen die Attribute des Punktes x_i in der i-ten Zeile haben


def getKNNData(matrixSignal, matrixBackground, lenTraining, lenSignal, lenBackground, shuffle = False, seed = 0):
    '''Takes two Matrizes matrixSignal and matrixBackground. Returns three numpy arrays:
    TrainingData contains 2*lenTraining points. TrainingData[0:lenTraining] is the background data used to train the knn classifier,
    TrainingData[lenTraining:2lenTraining] is the signal data used to train.
    TestDataSignal contains lenSignal signal points to test the model, TestDataBackground is analogous.
    If getKNNData is invoked with shuffle = True the given data is shuffled randomly with a function from sklearn.'''
    if shuffle == True:
        matrixSignal, matrixBackground = shuffle(matrixSignal, matrixBackground, random_state=seed)
    matrixSignal = matrixSignal[:lenTraining+lenSignal:,] # Nimm bitte die ersten lenTraining+lenSignal
    matrixBackground = matrixBackground[:lenTraining+lenBackground:,]
    
    trainingDataBackground, testDataBackground = np.split(matrixBackground, [lenTraining])
    trainingDataSignal, testDataSignal = np.split(matrixSignal, [lenTraining])
    #trainingDataBackground, testDataBackground = train_test_split(matrixBackground, test_size = lenBackground, train_size = lenTraining, shuffle=False)
    #trainingDataSignal, testDataSignal = train_test_split(matrixSignal, test_size = lenSignal, train_size = lenTraining, suffle=False)
    trainingData = np.concatenate((trainingDataBackground, trainingDataSignal), axis=0)
    
    return trainingData, testDataBackground, testDataSignal

In [5]:
lenTraining = 5000
lenSignal = 10000
lenBackground = 20000
k = 10

matrixSignal = getDataFromFile(filename='NeutrinoMC.hdf5', key='Signal')
matrixBackground = getDataFromFile(filename='NeutrinoMC.hdf5', key='Background')
trainingData, testDataBackground, testDataSignal = getKNNData(matrixSignal, matrixBackground,
                                                              lenTraining, lenSignal, lenBackground)
labels = np.concatenate((np.zeros(lenTraining),np.ones(lenTraining))) # 0 = Background, 1 = Signal

In [20]:
print(matrixSignal)
test = matrixSignal[0:1,]
print(test)
print(test.shape)
print(np.ravel(test))
print(np.ravel(test).shape)

[[ 26.           6.69494405   2.20707621]
 [ 38.           7.74364571   3.57920725]
 [ 87.           7.41714979   2.96454426]
 ..., 
 [ 52.           7.71972272   3.19118159]
 [ 25.           7.64833004   3.48405199]
 [ 12.           7.779087     3.8578863 ]]
[[ 26.           6.69494405   2.20707621]]
(1, 3)
[ 26.           6.69494405   2.20707621]
(3,)


In [7]:
def classifyData(k, trainingData, labels, testDataSignal, testDataBackground):
    '''Analyzes a given Testset based on trainingdata and labels. Uses knn with k nearest neighbors.
    Results (tp, fn, fp, tn, precision, recall and significance) are printed to screen.'''
    knn = KNN(k)
    knn.fit(trainingData, labels)
    predictionS = knn.predict(testDataSignal)
    predictionB = knn.predict(testDataBackground)
    predictionS = np.asarray(predictionS)
    predictionB = np.asarray(predictionB)
    tp = np.count_nonzero(predictionS == 1) # Neutrinos, die korrekt als Neutrino zugeordnet werden
    fn = np.count_nonzero(predictionS == 0) # Neutrinos, die fälschlicherweise als Background zugeordnet werden
    fp = np.count_nonzero(predictionB == 1) # Background, der fälschlicherweise als Neutrino zugeordnet wird
    tn = np.count_nonzero(predictionB == 0) # Background, der korrekt als Background zugeordnet wird
    print('tp', tp)
    print('fn', fn)
    print('fp', fp)
    print('tn', tn)
    Reinheit = tp/(tp+fp)
    Effizienz = tp/(tp+fn)
    Signifikanz = tp/(np.sqrt(tp+fp))
    print('Reinheit =', Reinheit)
    print('Effizienz =', Effizienz)
    print('Signifikanz =', Signifikanz)

In [8]:
classifyData(k, trainingData, labels, testDataSignal, testDataBackground)

KeyboardInterrupt: 

## Teilaufgabe e)
Wir verwenden nun $\mathrm{log}_{10}$(Hits) statt Hits. Das führt zu einer verbesserten Zuordnung der Daten, da alle Qualitätsmaße verbessert werden. Es wurden nur noch 1,2% statt 2,6% der Neutrinos fälschlicherweise dem Background zugeordnet.

In [ ]:
matrixSignal = getDataFromFile(filename='NeutrinoMC.hdf5', key='Signal', log=True)
matrixBackground = getDataFromFile(filename='NeutrinoMC.hdf5', key='Background', log=True)
trainingData, testDataBackground, testDataSignal = getKNNData(matrixSignal, matrixBackground,
                                                              lenTraining, lenSignal, lenBackground)

In [ ]:
classifyData(k, trainingData, labels, testDataSignal, testDataBackground)

## Teilaufgabe f)
Nun wird $k=20$ gewählt. Es ist zu erkennen, dass die Analyse ohne den Logarithmus streng schlechter ist.
Mit dem Logarithmus ist es schwieriger: Es werden mehr Neutrinos korrekt zugeordnet als mit $k=10$, ober die Zuordnung der Background-Events im Testset wird schlechter. Dies resultiert in einer geringeren Reinheit, aber in einer leicht besseren Effizienz.<br>
Man müsste weitere Werte für $k$ untersuchen, um genauere Schlüsse zu ziehen; jedoch kann man vermuten, dass in der Nähe von $k=20$ Underfitting vorliegt. Wenn zu viele nächste Nachbarn untersucht werden, wird die Analyse geglättet und unsensitiver für den Input (Beispiel: Wählt man zum Beispiel $k$ als Größe des Testsets wäre der Output konstant und unabhängig von den Attributwerten). Wie gesagt müsste man die Qualitätsmaße in Abhängigkeit von $k$ allerdings genauer untersuchen, um auch nur annäherend sichere Aussagen über die beste Wahl von $k$ zu treffen.

In [ ]:
k = 20

In [ ]:
matrixSignal = getDataFromFile(filename='NeutrinoMC.hdf5', key='Signal')
matrixBackground = getDataFromFile(filename='NeutrinoMC.hdf5', key='Background')
trainingData, testDataBackground, testDataSignal = getKNNData(matrixSignal, matrixBackground,
                                                              lenTraining, lenSignal, lenBackground)
labels = np.concatenate((np.zeros(lenTraining),np.ones(lenTraining))) # 0 = Background, 1 = Signal
classifyData(k, trainingData, labels, testDataSignal, testDataBackground)

In [ ]:
matrixSignal = getDataFromFile(filename='NeutrinoMC.hdf5', key='Signal', log=True)
matrixBackground = getDataFromFile(filename='NeutrinoMC.hdf5', key='Background', log=True)
trainingData, testDataBackground, testDataSignal = getKNNData(matrixSignal, matrixBackground,
                                                              lenTraining, lenSignal, lenBackground)
classifyData(k, trainingData, labels, testDataSignal, testDataBackground)